In [1]:
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np
import datetime
import imutils
import time
import dlib
import cv2

In [2]:
#Using dlib's pre trained face and shape detector to localise face in the image and then other facial features
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/home/amit/driver1/facial-landmarks/shape_predictor_68_face_landmarks.dat')
#In the above line, use the address at which you saved the shape_predictor file

In [ ]:
vs = VideoStream(0).start()
#time.sleep(2.0)

while True:

    frame = vs.read()
    time.sleep(1.0)
    #frame = imutils.resize(frame, width=400)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
    
    #Extracting left and right eye images from face
    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        for (name, (j, k)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
            (x, y, w, h) = cv2.boundingRect(np.array([shape[j:k]]))
            roi = gray[y-10:y + h+10, x-10:x + w+10]
            #roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
            if(j==36):
                right_eye = roi
            elif(j==42):
                left_eye = roi
                
    cv2.imwrite('right_eye.jpg', right_eye)
    right_eye = cv2.imread('right_eye.jpg',0)
        
    equ = cv2.equalizeHist(right_eye)
    hist, bin_edges = np.histogram(equ.flatten(), 256, [0,256])
    pupil_pixels = equ.size*0.08
    threshold_fake = 0
    for i in range(0, 256):
        threshold_fake = threshold_fake+hist[i]
        if(threshold_fake>pupil_pixels):
            break
        
    threshold1 = i
    
    new_img = np.zeros((equ.shape[0],equ.shape[1]))
    for i in range(10, equ.shape[0]-10):
        for j in range(10, equ.shape[1]-10):
            if equ[i,j] < threshold1:
                new_img[i,j] = 255
            else:
                new_img[i,j] = 0
                
    kernel = np.ones((2,2), np.uint8)
    new_img_er1 = cv2.erode(new_img, kernel, iterations=1)           
    
    new_img_er1 = np.uint8(new_img_er1)
    im2, contours, hierarchy = cv2.findContours(new_img_er1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    if(not contours):
        continue
    right_eye2 = right_eye
    c = max(contours, key = cv2.contourArea)
    M = cv2.moments(c)

    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
    else:
        cX, cY = 0, 0
    cv2.circle(right_eye2, (cX, cY), 2, (255, 255, 255), -1)
    cv2.drawContours(new_img_er1, [c], -1, (255, 255, 255), 2)
    
    if(cX!=0 | cY!=0):
        cv2.imshow("right_eye2", right_eye2)
        cv2.imshow("new_img_er1", new_img_er1)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
    
    key = cv2.waitKey(1) & 0xFF
if (key == ord("q")):
    break  
        
        
cv2.destroyAllWindows()
vs.stop()